# Vehicle Search Agent API Testing

Interactive notebook for testing the API endpoints.

**Prerequisites:** Make sure the API server is running:
```bash
python api_server.py
```

In [ ]:
import requests
import json
from pprint import pprint

BASE_URL = "http://localhost:8000"

## 1. Health Check

In [ ]:
response = requests.get(f"{BASE_URL}/")
print(f"Status: {response.status_code}")
pprint(response.json())

## 2. Start a Conversation

In [ ]:
# Send first message
response = requests.post(
    f"{BASE_URL}/chat",
    json={
        "message": "I want a Jeep around $30k in Colorado"
    }
)

data = response.json()
session_id = data.get('session_id')

print(f"Session ID: {session_id}")
print(f"Mode: {data.get('mode')}")
print(f"\nAgent Response:")
print(data.get('response'))
print(f"\nVehicles Found: {len(data.get('vehicles', []))}")

## 3. View Filters

In [ ]:
print("Current Filters:")
pprint(data.get('filters'))

print("\nInferred Preferences:")
pprint(data.get('preferences'))

## 4. View Vehicle Listings

In [ ]:
vehicles = data.get('vehicles', [])

for i, vehicle in enumerate(vehicles[:5], 1):  # Show top 5
    v = vehicle.get('vehicle', {})
    listing = vehicle.get('retailListing', {})
    
    print(f"#{i}. {v.get('year')} {v.get('make')} {v.get('model')} {v.get('trim', '')}")
    print(f"    Price: ${listing.get('price', 'N/A'):,}")
    print(f"    Miles: {listing.get('miles', 'N/A'):,}")
    print(f"    Location: {listing.get('city')}, {listing.get('state')}")
    print(f"    VIN: {v.get('vin')}")
    print()

## 5. Continue Conversation (Analytical Mode)

In [ ]:
# Ask about a specific vehicle
response = requests.post(
    f"{BASE_URL}/chat",
    json={
        "message": "Tell me more about #1",
        "session_id": session_id
    }
)

data = response.json()

print(f"Mode: {data.get('mode')}")
print(f"\nAgent Response:")
print(data.get('response'))

## 6. Get Session State

In [ ]:
response = requests.get(f"{BASE_URL}/session/{session_id}")
session_data = response.json()

print("Session State:")
print(f"  Filters: {session_data.get('filters')}")
print(f"  Preferences: {session_data.get('preferences')}")
print(f"  Vehicles: {len(session_data.get('vehicles', []))}")
print(f"  Conversation Length: {len(session_data.get('conversation_history', []))} messages")

## 7. View Conversation History

In [ ]:
print("Conversation History:")
print("=" * 70)

for msg in session_data.get('conversation_history', []):
    role = msg.get('role', 'unknown').upper()
    content = msg.get('content', '')
    
    if role == 'USER':
        print(f"\n👤 User: {content}")
    else:
        print(f"\n🤖 Agent: {content[:200]}...")  # Truncate long responses
    print("-" * 70)

## 8. Get Vehicle Details by VIN

In [ ]:
# Get VIN from first vehicle
if vehicles:
    vin = vehicles[0].get('vehicle', {}).get('vin')
    
    if vin:
        response = requests.get(f"{BASE_URL}/vehicles/{vin}")
        vehicle_details = response.json()
        
        print("Vehicle Details:")
        pprint(vehicle_details.get('vehicle', {}))
        
        print(f"\nPhotos: {len(vehicle_details.get('photos', []))} available")
else:
    print("No vehicles available to query")

## 9. Custom Message (Try Your Own!)

In [ ]:
# Try your own message
custom_message = "What's the safety rating of #1?"  # Change this!

response = requests.post(
    f"{BASE_URL}/chat",
    json={
        "message": custom_message,
        "session_id": session_id
    }
)

data = response.json()
print(f"Your Message: {custom_message}")
print(f"\nAgent Response:")
print(data.get('response'))

## 10. Reset Session

In [ ]:
response = requests.post(
    f"{BASE_URL}/session/reset",
    json={"session_id": session_id}
)

print("Session reset:")
pprint(response.json())

## 11. List All Active Sessions

In [ ]:
response = requests.get(f"{BASE_URL}/sessions")
sessions_data = response.json()

print(f"Active Sessions: {sessions_data.get('active_sessions')}")
print(f"Session IDs: {sessions_data.get('session_ids')}")